In [1]:
import numpy as np
import pandas as pd
import glob
import os
from scipy.stats import pearsonr
import anndata as ad
from scipy.sparse import csr_matrix
from scipy.stats import zscore
import time
from scipy.sparse import issparse
import re


In [2]:
atac_path = '/mnt/c/Users/donna/Downloads/Thesis/data/test/atac_files_test'

In [3]:
bed_df = pd.read_csv("/mnt/c/Users/donna/Downloads/Thesis/TSSClassification/beds/dts_beds/atac_match.bed", sep='\t', header=None)

In [4]:
gene_names_in_bed = set(bed_df[3].unique())

In [6]:
for my_file in os.listdir(atac_path):
    if not my_file.endswith('.csv'):
        continue
    tissue_data = pd.read_csv(os.path.join(atac_path, my_file))
    parts = my_file.split('_')
    parts[-1] = parts[-1].replace('.csv', '')
    print(parts)
    if (len(parts) - 1) > 3:
        tissue = parts[2] + '_' + parts[3]
    else:
        tissue = parts[2]
    print(tissue)  

    ##Filteren op matching genes
    matching_cols = [col for col in tissue_data.columns if col in gene_names_in_bed]
    tissue_data = tissue_data[matching_cols]

    for types_file in os.listdir(atac_path):
        if not types_file.endswith('.npy'):
            continue
        npy_parts = re.split(r'atac_nearest_cell_types_|_harmony.npy', types_file)
        print(npy_parts)
        if len(npy_parts) > 3:
            npy_tissue = npy_parts[1] + '_' + npy_parts[2]
        else:
            npy_tissue = npy_parts[1]
            print(npy_tissue)
        if npy_tissue == tissue:
            cell_types = np.load(os.path.join(atac_path, types_file))
            print(cell_types)
            num_cols = tissue_data.select_dtypes(include='number')
            num_cols['cell_type'] = cell_types
            num_cols = num_cols.groupby('cell_type').mean()
            print(num_cols)

            num_cols.to_csv(f'/mnt/c/Users/donna/Downloads/Thesis/data/test/{tissue}.csv', index=True)



['tissue', 'subset', 'uterus']
uterus
['', 'uterus', '']
uterus
['fibroblast' 'fibroblast' 'vascular associated smooth muscle cell' ...
 'fibroblast' 'fibroblast' 'fibroblast']
                                            A1BG      A1CF       A2M  \
cell_type                                                              
CD4-positive, alpha-beta T cell         0.000000  0.000000  0.111111   
CD8-positive, alpha-beta T cell         0.000000  0.000000  0.053030   
ciliated epithelial cell                0.000000  0.066667  0.000000   
endothelial cell                        0.000000  0.003914  0.083170   
epithelial cell of uterus               0.000000  0.025424  0.033898   
fibroblast                              0.005081  0.008092  0.058525   
macrophage                              0.000000  0.029851  0.059701   
mast cell                               0.000000  0.000000  0.285714   
mature NK T cell                        0.000000  0.000000  0.000000   
myometrial cell                